# Pydantic AI Walkthrough

Inspired bij: https://ai.pydantic.dev/agents/

Setup the environment

In [1]:
import dotenv
import os
import logfire

import tool_functions as tf

import nest_asyncio

nest_asyncio.apply()

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")
logfire_token = os.getenv("LOGFIRE_TOKEN")

# configure logfire
logfire.configure(token=logfire_token)
logfire.instrument_pydantic_ai()

Logfire project URL: ]8;id=167810;https://logfire-eu.pydantic.dev/elbrink/claude-test\https://logfire-eu.pydantic.dev/elbrink/claude-test]8;;\


1 Definieer een basis agent

In [2]:
from pydantic_ai import Agent
from pydantic_ai.models.anthropic import AnthropicModel

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = AnthropicModel(os.getenv("DEFAULT_MODEL"))

basic_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)

Stel een vraag aan de agent

In [3]:
result = basic_agent.run_sync("geef een defintie van een agent")
print(result.output)

12:32:51.671 basic_agent run
12:32:51.671   chat claude-3-5-haiku-20241022
Hier is een definitie van een agent:

Een agent is een systeem dat zich in een omgeving bevindt en dat autonoom kan waarnemen, redeneren en handelen om bepaalde doelen te bereiken of taken uit te voeren. Belangrijke kenmerken van een agent zijn:

1. Autonomie: Het kan zelfstandig beslissingen nemen
2. Waarneming: Het kan informatie verzamelen uit zijn omgeving
3. Redenatie: Het kan logisch redeneren en keuzes maken
4. Actie: Het kan concrete handelingen uitvoeren om doelen te bereiken

Agents kunnen verschillende vormen aannemen:
- Softwareagenten (zoals AI-systemen)
- Fysieke agenten (zoals robots)
- Virtuele agenten (zoals chatbots)

In de artificiële intelligentie wordt een agent gezien als een systeem dat:
- Zinvolle beslissingen kan nemen
- Kan leren van eerdere ervaringen
- Doelgericht werkt
- Kan communiceren met andere systemen of mensen

Kortom, een agent is een intelligent systeem dat zelfstandig kan w

Definieer tools voor de agent

In [4]:
tool_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)


@tool_agent.tool_plain
def tool_bereken_btw(bedrag: float, btw_percentage: float):
    """
    Bereken BTW over een bedrag.
    """
    return tf.calculate_btw(bedrag, btw_percentage)


@tool_agent.tool_plain
def tool_bereken_korting(bedrag: float, korting_percentage: float):
    """
    Bereken korting over een bedrag.
    """
    return tf.calculate_discount(bedrag, korting_percentage)


@tool_agent.tool_plain
def tool_get_inwoners_gemeente(gemeente_naam: str, jaar: str):
    """
    Haal inwonersaantallen op van Nederlandse gemeenten via CBS API.
    """
    return tf.get_inwoners_gemeente(gemeente_naam, jaar)

Test een aantal tool calls

In [5]:
result2 = tool_agent.run_sync(
    "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld"
)
print(result2.output)

12:34:11.304 tool_agent run
12:34:11.304   chat claude-3-5-haiku-20241022
12:34:14.354   running 1 tool
12:34:14.355     running tool: tool_bereken_btw
12:34:14.356   chat claude-3-5-haiku-20241022
12:34:16.948   running 1 tool
12:34:16.948     running tool: tool_bereken_korting
12:34:16.950   chat claude-3-5-haiku-20241022
Hier is een duidelijk overzicht:
1. Oorspronkelijke prijs exclusief BTW: €100,00
2. BTW (21%): €21,00
3. Prijs inclusief BTW: €121,00
4. Korting (15%): €18,15
5. Uiteindelijk te betalen bedrag: €102,85

Je koopt het product voor €100,00 exclusief BTW. Na toevoeging van 21% BTW wordt dat €121,00. Vervolgens krijg je 15% korting over dit bedrag, wat neerkomt op €18,15 korting. Je eindprijs wordt dus €102,85.


In [6]:
result = tool_agent.run_sync(
    "Geef een overzicht van de top-7 gemeenten in Nederland met het grootste aantal inwoners in 2024. Laat ook het aantal inwoners van Groningen zien."
)
print(result.output)

12:35:08.835 tool_agent run
12:35:08.836   chat claude-3-5-haiku-20241022
12:35:16.122   running 7 tools
12:35:16.122     running tool: tool_get_inwoners_gemeente
12:35:16.122     running tool: tool_get_inwoners_gemeente
12:35:16.123     running tool: tool_get_inwoners_gemeente
12:35:16.123     running tool: tool_get_inwoners_gemeente
12:35:16.123     running tool: tool_get_inwoners_gemeente
12:35:16.123     running tool: tool_get_inwoners_gemeente
12:35:16.123     running tool: tool_get_inwoners_gemeente
12:35:17.692   chat claude-3-5-haiku-20241022
Op basis van de verkregen gegevens kan ik de top-7 gemeenten in Nederland met het grootste aantal inwoners in 2024 als volgt presenteren:

1. Amsterdam: 931.298 inwoners
2. Rotterdam: 670.610 inwoners
3. Utrecht: 374.238 inwoners
4. Eindhoven: 246.417 inwoners
5. Groningen: 243.768 inwoners
6. Tilburg: 229.836 inwoners

Ik kon helaas geen gegevens vinden voor Den Haag, dus de top-7 is niet volledig. De gemeente Groningen heeft 243.768 inwo

In [ ]:
result = tool_agent.run_sync(
    "Als alle inwoners uit Eindhoven in 2025 15 euro betalen, hoeveel krijg ik dan."
)
print(result.output)

Voeg geheugen toe aan de agent

In [ ]:
mem_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)

result = mem_agent.run_sync(
    "wat was ook alweer het btw bedrag van het eerder gekochte product X en hoeveel korting kreeg ik. Zoek dit uit de conversatie hiervoor.",
    message_history=result2.all_messages(),
)
print(result.output)

PyDantic MCP client

In [8]:
import asyncio
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerSSE

server = MCPServerSSE(url="http://localhost:8050/sse")
mcp_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
    mcp_servers=[server],
)

In [ ]:
async def run_query():
    async with mcp_agent.run_mcp_servers():
        print(await server.list_tools())
        result = await mcp_agent.run(
            "Geef een overzicht van de top-10 gemeenten in Nederland met het grootste aantal inwoners in 2024."
        )
        print(result.output)
        result = await mcp_agent.run(
            "Hoeveel inwoners had Rotterdam meer dan Eindhoven in 2024?"
        )
        print(result.output)


asyncio.run(run_query())

[ToolDefinition(name='calculate_btw', description='\n    Bereken BTW over een bedrag.\n\n    Args:\n        bedrag: Het bedrag exclusief BTW\n        btw_percentage: BTW percentage (standaard 21%)\n\n    Returns:\n        Dict met BTW berekening resultaten\n    ', parameters_json_schema={'properties': {'bedrag': {'title': 'Bedrag', 'type': 'number'}, 'btw_percentage': {'default': 21.0, 'title': 'Btw Percentage', 'type': 'number'}}, 'required': ['bedrag'], 'title': 'calculate_btwArguments', 'type': 'object'}), ToolDefinition(name='calculate_discount', description='\n    Bereken korting over een bedrag.\n    ', parameters_json_schema={'properties': {'bedrag': {'title': 'Bedrag', 'type': 'number'}, 'korting_percentage': {'title': 'Korting Percentage', 'type': 'number'}}, 'required': ['bedrag', 'korting_percentage'], 'title': 'calculate_discountArguments', 'type': 'object'}), ToolDefinition(name='get_inwoners_gemeente', description='\n    Geef het aantal inwoners van een gemeente.\n    ', 